In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")



In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AvgPrice FROM home_sales WHERE Bedrooms = 4 GROUP BY Year ORDER BY Year")
result.show()


+----+---------+
|Year| AvgPrice|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
from pyspark.sql.functions import year, to_date

result = spark.sql("SELECT YEAR(to_date(CAST(date_built AS STRING))) AS YearBuilt, ROUND(AVG(Price), 2) AS AvgPrice FROM home_sales WHERE Bedrooms = 3 AND Bathrooms = 3 GROUP BY YearBuilt ORDER BY YearBuilt")
result.show()


+---------+---------+
|YearBuilt| AvgPrice|
+---------+---------+
|     2010|292859.62|
|     2011|291117.47|
|     2012|293683.19|
|     2013|295962.27|
|     2014|290852.27|
|     2015| 288770.3|
|     2016|290555.07|
|     2017|292676.79|
+---------+---------+



In [13]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = spark.sql("SELECT YEAR(to_date(CAST(date_built AS STRING))) AS YearBuilt, ROUND(AVG(Price), 2) AS AvgPrice " +
                   "FROM home_sales " +
                   "WHERE Bedrooms = 3 AND Bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 " +
                   "GROUP BY YearBuilt " +
                   "ORDER BY YearBuilt")

result.show()


+---------+---------+
|YearBuilt| AvgPrice|
+---------+---------+
|     2010|285010.22|
|     2011|276553.81|
|     2012|307539.97|
|     2013|303676.79|
|     2014|298264.72|
|     2015|297609.97|
|     2016| 293965.1|
|     2017|280317.58|
+---------+---------+



In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

import time

start_time = time.time()

result = spark.sql("SELECT AVG(view) AS AvgViewRating FROM home_sales WHERE Price >= 350000")
result.show()

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))


+-------------+
|AvgViewRating|
+-------------+
|       32.264|
+-------------+

--- 0.7306861877441406 seconds ---


In [15]:
# 7. Cache the the temporary table home_sales.

spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Enable caching
spark.sql("CACHE TABLE home_sales")

# Measure runtime with caching
start_time = time.time()

result = spark.sql("SELECT AVG(view) AS AvgViewRating FROM home_sales WHERE Price >= 350000")
result.show()

end_time = time.time()

print("Runtime with caching: %s seconds" % (end_time - start_time))

# Uncache the table
spark.sql("UNCACHE TABLE home_sales")

# Measure runtime without caching
start_time = time.time()

result = spark.sql("SELECT AVG(view) AS AvgViewRating FROM home_sales WHERE Price >= 350000")
result.show()

end_time = time.time()

print("Runtime without caching: %s seconds" % (end_time - start_time))


+-------------+
|AvgViewRating|
+-------------+
|       32.264|
+-------------+

Runtime with caching: 0.7283227443695068 seconds
+-------------+
|AvgViewRating|
+-------------+
|       32.264|
+-------------+

Runtime without caching: 0.7180581092834473 seconds


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "path_to_output_directory"
df.write.partitionBy("date_built").parquet(output_path)

In [22]:
# 11. Read the parquet formatted data.
parquet_dir = "path_to_output_directory"

parquet_data = spark.read.parquet(parquet_dir)

In [23]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_table")

In [25]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

import time

# Enable caching for the Parquet DataFrame
parquet_data.createOrReplaceTempView("parquet_table")
spark.sql("CACHE TABLE parquet_table")

# Measure runtime with caching
start_time = time.time()

result = spark.sql("SELECT AVG(view) AS AvgViewRating FROM parquet_table WHERE Price >= 350000")
result.show()

end_time = time.time()

print("Runtime with caching: %s seconds" % (end_time - start_time))

# Uncache the Parquet DataFrame
spark.sql("UNCACHE TABLE parquet_table")

# Measure runtime without caching
start_time = time.time()

result = spark.sql("SELECT AVG(view) AS AvgViewRating FROM parquet_table WHERE Price >= 350000")
result.show()

end_time = time.time()

print("Runtime without caching: %s seconds" % (end_time - start_time))


+-------------+
|AvgViewRating|
+-------------+
|       32.264|
+-------------+

Runtime with caching: 0.5049283504486084 seconds
+-------------+
|AvgViewRating|
+-------------+
|       32.264|
+-------------+

Runtime without caching: 0.6755514144897461 seconds


In [26]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [27]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

if is_cached:
    print("The 'home_sales' temporary table is still cached.")
else:
    print("The 'home_sales' temporary table is no longer cached.")

The 'home_sales' temporary table is no longer cached.
